# Clustering Crypto

In [177]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [178]:
# Load the crypto_data.csv dataset.
df = pd.read_csv("Resources/crypto_data.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1247,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [179]:
# Keep all the cryptocurrencies that are being traded.
df = df[df["IsTrading"] == True]


In [180]:
# Keep all the cryptocurrencies that have a working algorithm.
for col in df.columns:
    print(f"Column {col} has {df[col].isnull().sum()} null values.")

Column CoinName has 0 null values.
Column Algorithm has 0 null values.
Column IsTrading has 0 null values.
Column ProofType has 0 null values.
Column TotalCoinsMined has 459 null values.
Column TotalCoinSupply has 0 null values.


In [181]:
# Remove the "IsTrading" column. 
df.drop("IsTrading", axis=1, inplace=True)


C:\Users\Alec\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [182]:
# Remove rows that have at least 1 null value.
df.dropna(inplace=True)

C:\Users\Alec\AppData\Roaming\Python\Python37\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [183]:
# Keep the rows where coins are mined.
df = df[df["TotalCoinsMined"] > 0]


In [184]:
# Create a new DataFrame that holds only the cryptocurrencies names.
Crypto_names = pd.DataFrame(df["CoinName"])
Crypto_names

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum
...,...
1238,ZEPHYR
1242,Gapcoin
1245,Beldex
1246,Horizen


In [185]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df.drop("CoinName", axis=1, inplace=True)
df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
1238,SHA-256,DPoS,2.000000e+09,2000000000
1242,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,CryptoNight,PoW,9.802226e+08,1400222610
1246,Equihash,PoW,7.296538e+06,21000000


In [186]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(df, columns=["Algorithm", "ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [187]:
# Standardize the data with StandardScaler().
X = StandardScaler().fit_transform(X)
X

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [188]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

X_pca = pca.fit_transform(X)

In [189]:
# Create a DataFrame with the three principal components.
pca_df = pd.DataFrame(data=X_pca, columns = ["PC 1", "PC 2", "PC 3"], index = df.index)

pca_df.head()

,PC 1,PC 2,PC 3
0,-0.344073,1.011379,-0.601946
2,-0.327349,1.011658,-0.602350
5,2.313881,1.712877,-0.600914
7,-0.148538,-1.311359,0.212134
8,-0.151859,-2.016688,0.370351


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [190]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Loop through each K value, find inertia, and store in list
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)


elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Users\Alec\Downloads\Anaconda\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [191]:
# Initialize the K-Means model.
model = KMeans(n_clusters=5, random_state=0)

# Fit the model
model.fit(pca_df)

# Predict clusters
model.predict(pca_df)

array([0, 0, 4, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 4,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 4, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 4, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 4, 1, 0, 1, 0, 1, 0,

In [192]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df, pca_df], axis=1, sort=False)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"]=Crypto_names["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.344073,1.011379,-0.601946,42 Coin,0
2,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.327349,1.011658,-0.602350,404Coin,0
5,X13,PoW/PoS,2.927942e+10,314159265359,2.313881,1.712877,-0.600914,EliteCoin,4
7,SHA-256,PoW,1.792718e+07,21000000,-0.148538,-1.311359,0.212134,Bitcoin,1
8,Ethash,PoW,1.076842e+08,0,-0.151859,-2.016688,0.370351,Ethereum,1
9,Scrypt,PoW,6.303924e+07,84000000,-0.170297,-1.139874,-0.032035,Litecoin,1
10,X11,PoW/PoS,9.031294e+06,22000000,-0.393314,1.239046,-0.503817,Dash,0
11,CryptoNight-V7,PoW,1.720114e+07,0,-0.146917,-2.191822,0.333533,Monero,1
12,Ethash,PoW,1.133597e+08,210000000,-0.150293,-2.016773,0.370338,Ethereum Classic,1
13,Equihash,PoW,7.383056e+06,21000000,-0.130968,-2.036229,0.395633,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [193]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
x="PC 1",
    y="PC 2",
    z="PC 3",
color="Class",
    symbol="Class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [194]:
# Create a table with tradable cryptocurrencies.
tradable_df = clustered_df[["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"]]
tradable_df.hvplot.table()

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [195]:
# Print the total number of tradable cryptocurrencies.
print(len(tradable_df))

532


In [196]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
minmaxscaler = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
print(minmaxscaler[0:5])

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 [2.10000000e-05 1.81084216e-05]
 [0.00000000e+00 1.08773140e-04]]


In [197]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
new_df = pd.DataFrame(
    data=minmaxscaler, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
new_df["CoinName"]=Crypto_names["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
new_df["Class"] = model.labels_
new_df.head(10)


,TotalCoinSupply,TotalCoinsMined,CoinName,Class
0,4.200000e-11,0.000000,42 Coin,0
2,5.320000e-04,0.001066,404Coin,0
5,3.141593e-01,0.029576,EliteCoin,4
7,2.100000e-05,0.000018,Bitcoin,1
8,0.000000e+00,0.000109,Ethereum,1
9,8.400000e-05,0.000064,Litecoin,1
10,2.200000e-05,0.000009,Dash,0
11,0.000000e+00,0.000017,Monero,1
12,2.100000e-04,0.000115,Ethereum Classic,1
13,2.100000e-05,0.000007,ZCash,1


In [198]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
new_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["Class"],
    by="Class",
)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,Class)